In [141]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

In [142]:
# Loading Dataset for Sales in Conneticut to Pandas
df = pd.read_csv('data\Real_Estate_Sales_2001-2022_GL.csv')

<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\yulia\AppData\Local\Temp\ipykernel_13000\850320232.py:2: SyntaxWarning: invalid escape sequence '\R'
  df = pd.read_csv('data\Real_Estate_Sales_2001-2022_GL.csv')
C:\Users\yulia\AppData\Local\Temp\ipykernel_13000\850320232.py:2: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data\Real_Estate_Sales_2001-2022_GL.csv')


In [143]:
# Checking for missing values
df.isna().sum()

Serial Number             0
List Year                 0
Date Recorded             2
Town                      0
Address                  51
Assessed Value            0
Sale Amount               0
Sales Ratio               0
Property Type        382446
Residential Type     398389
Non Use Code         784178
Assessor Remarks     926401
OPM remarks         1084598
Location             799518
dtype: int64

In [144]:
# Dropping colums that won't be useful for modeling
df_clean = df.drop(columns = ['Serial Number', 'OPM remarks', 'Assessor Remarks', 'Non Use Code'])

In [145]:
# Dropping transactions with no sale amount
df_clean = df_clean[df_clean['Sale Amount'] != 0]

In [146]:
# Seeing current state of database
df_clean.describe()

,List Year,Assessed Value,Sale Amount,Sales Ratio
count,1.095819e+06,1.095819e+06,1.095819e+06,1.095819e+06
mean,2.011231e+03,2.819331e+05,4.059840e+05,9.619789e+00
std,6.770525e+00,1.658927e+06,5.147712e+06,1.803151e+03
min,2.001000e+03,0.000000e+00,1.000000e+00,0.000000e+00
25%,2.005000e+03,8.915000e+04,1.450000e+05,4.785714e-01
50%,2.011000e+03,1.406300e+05,2.340000e+05,6.110000e-01
75%,2.018000e+03,2.283400e+05,3.750000e+05,7.711000e-01
max,2.022000e+03,8.815100e+08,5.000000e+09,1.226420e+06


In [147]:
# Checking for missing values again
df_clean.isna().sum()

List Year                0
Date Recorded            0
Town                     0
Address                 24
Assessed Value           0
Sale Amount              0
Sales Ratio              0
Property Type       380748
Residential Type    396691
Location            798031
dtype: int64

In [148]:
# Check the unique property types in the dataset. 
unique_values_property_type = df_clean['Property Type'].unique()
print(unique_values_property_type)

['Residential' 'Commercial' 'Vacant Land' 'Public Utility' 'Apartments'
 nan 'Industrial' 'Condo' 'Two Family' 'Single Family' 'Three Family'
 'Four Family']


In [149]:
# Count how many times each sales ratio appears in the dataset. How many properties sold for exactly the assessed price?
price_to_sell = df_clean['Sales Ratio'].value_counts()

In [150]:
# Fill missing 'Property Type' values with 'unknown'
df_clean['Property Type'].fillna('unknown', inplace=True)

C:\Users\yulia\AppData\Local\Temp\ipykernel_13000\1692148295.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Property Type'].fillna('unknown', inplace=True)


In [151]:
df_clean.isna().sum()

List Year                0
Date Recorded            0
Town                     0
Address                 24
Assessed Value           0
Sale Amount              0
Sales Ratio              0
Property Type            0
Residential Type    396691
Location            798031
dtype: int64

In [152]:
# Fill missing 'Residential Type' values with 'non-residential'
df_clean['Residential Type'].fillna('non-residential', inplace=True)

C:\Users\yulia\AppData\Local\Temp\ipykernel_13000\25978290.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Residential Type'].fillna('non-residential', inplace=True)


In [153]:
df_clean.isna().sum()

List Year                0
Date Recorded            0
Town                     0
Address                 24
Assessed Value           0
Sale Amount              0
Sales Ratio              0
Property Type            0
Residential Type         0
Location            798031
dtype: int64

In [154]:
# Fill missing 'Address' values with 'unknown'
df_clean.fillna({'Address': 'unknown'}, inplace=True)

In [155]:
df_clean.isna().sum()

List Year                0
Date Recorded            0
Town                     0
Address                  0
Assessed Value           0
Sale Amount              0
Sales Ratio              0
Property Type            0
Residential Type         0
Location            798031
dtype: int64

In [156]:
# Convert 'Date Recorded' to datetime format to make use of month value
df_clean['Date Recorded'] = pd.to_datetime(df_clean['Date Recorded'])

In [157]:
# Create a function to categorize sales into seasons based on the month
def season_of_sale(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'
    
# Apply the function to create a new 'season_sold' column
df_clean['season_sold'] = df_clean['Date Recorded'].dt.month.apply(season_of_sale)

In [158]:
# Print column names to verify the new feature was added
print(df_clean.columns)

Index(['List Year', 'Date Recorded', 'Town', 'Address', 'Assessed Value',
       'Sale Amount', 'Sales Ratio', 'Property Type', 'Residential Type',
       'Location', 'season_sold'],
      dtype='object')


In [159]:
# Count the number of sales in each season. This is definitely useful, more properties are sold in summer
season_count = df_clean['season_sold'].value_counts()
print(season_count)

season_sold
Summer    335963
Fall      278699
Spring    259308
Winter    221849
Name: count, dtype: int64


In [160]:
# Checking top sales.
top_100_sales = df_clean.nlargest(100, 'Sale Amount')
print(top_100_sales)

        List Year Date Recorded        Town                      Address  \
59788        2020    2021-08-19  Willington             456 TOLLAND TPKE   
799678       2016    2016-12-06    Stamford             115 TOWNE STREET   
801991       2016    2016-12-06    Stamford         200 HENRY STREET # 3   
825265       2016    2016-12-06    Stamford         200 HENRY STREET # 4   
835261       2016    2016-12-06    Stamford             120 TOWNE STREET   
...           ...           ...         ...                          ...   
30352        2020    2021-01-15    Stamford       93 GLENBROOK ROAD #105   
32960        2020    2021-01-15    Stamford       93 GLENBROOK ROAD #205   
33538        2020    2021-01-15    Stamford       93 GLENBROOK ROAD #101   
34802        2020    2021-01-15    Stamford       93 GLENBROOK ROAD #201   
38016        2020    2021-01-15    Stamford  93 GLENBROOK ROAD GARAGE #2   

        Assessed Value   Sale Amount  Sales Ratio Property Type  \
59788        2238530

In [161]:
# When seeing described database, some high sales were alarming. Checking highest and lowest.
min_saleprice = df_clean['Sale Amount'].min()
max_saleprice = df_clean['Sale Amount'].max()
print(min_saleprice)
print(max_saleprice)
# Definitely concerning. Queuing iup to resolve.


1.0
5000000000.0


In [162]:
# Show the lowest 100 sales by 'Sale Amount'
lowest_100_sales = df_clean.nsmallest(100, 'Sale Amount')
print(lowest_100_sales)

        List Year Date Recorded        Town            Address  \
131300       2002    2002-11-12    Hartford      57 GILLETT ST   
194436       2003    2003-12-30    Brooklyn        HARTFORD RD   
205779       2003    2004-07-22     Bristol    66 EMMETT ST 12   
214135       2003    2004-03-19    Thompson  0 & 12 JOHNSON ST   
217960       2003    2004-03-19    Thompson  0 & 12 JOHNSON ST   
...           ...           ...         ...                ...   
869474       2017    2017-11-14     Norwalk      LEDGEBROOK DR   
67068        2002    2003-08-11      Oxford   MAPLE TREE HL RD   
432254       2006    2007-03-26    Coventry      BEECHWOOD TRL   
243195       2004    2005-06-21  East Haven       50 VENICE PL   
310429       2004    2005-09-12      Monroe    115 BOOTH HL ST   

        Assessed Value  Sale Amount    Sales Ratio Property Type  \
131300        165060.0          1.0  165060.000000       unknown   
194436           490.0          1.0     490.000000       unknown   
205

In [163]:
# Define lower and upper bounds for what is considered absurdly low or high sales
low_absurd = 0.01
high_absurd = 0.999

# Calculate the lower and upper bounds based on quantiles of the 'Sale Amount' column
lower_bound = df_clean['Sale Amount'].quantile(low_absurd)
upper_bound = df_clean['Sale Amount'].quantile(high_absurd)

# Print the absurdly low and high values
print(f"Absurdly low: {lower_bound}")
print(f"Absurdly high: {upper_bound}")

Absurdly low: 10400.0
Absurdly high: 13650000.0


In [164]:
# Filter the dataset to remove absurdly low or high sales
df_clean_filtered = df_clean[(df_clean['Sale Amount'] >= lower_bound) & (df_clean['Sale Amount'] <= upper_bound)]

# Check how sale amounts look after filtering
print(df_clean_filtered.shape)
print(df_clean_filtered['Sale Amount'].describe())

(1083833, 11)
count    1.083833e+06
mean     3.697899e+05
std      6.224099e+05
min      1.040000e+04
25%      1.480000e+05
50%      2.350000e+05
75%      3.790000e+05
max      1.365000e+07
Name: Sale Amount, dtype: float64


In [165]:
# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=['Property Type', 'Town', 'Property Type'], drop_first=True)

In [166]:
# Apply scaling to the numerical features
scaler = StandardScaler()
df_encoded[num_features] = scaler.fit_transform(df_encoded[num_features])

In [167]:
# Define categorical and numerical features for scaling and modeling. Not too many numerical features left but we work with what we have
cat_features = df_encoded.columns[df_encoded.columns.str.startswith('Property Type_') | df_encoded.columns.str.startswith('Residential Type_') | df_encoded.columns.str.startswith('season_sold_')].tolist()
num_features = ['Assessed Value', 'Sales Ratio']

In [168]:
# Define feature set and target variable
features = cat_features + num_features
target = 'Sale Amount'

In [169]:
# Split the data into training and testing sets (80% train, 20% test)
X = df_encoded[features]
y = df_encoded[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [170]:
# Define multiple regression models to try
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

In [171]:
# Loop through each model, train it, and evaluate its performance
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Evaluate performance with RMSE and R2 score
    rmse = mean_squared_error(y_test, y_pred, squared=False) 
    r2 = r2_score(y_test, y_pred)  
    
    # Print the model's name and its performance
    print(f"{name}: RMSE = {rmse}, R² = {r2}")

d:\dev\REAL-ESTATE-EDA-AND-MODEL\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear Regression: RMSE = 1660009.4344344118, R² = 0.1653231610968654


d:\dev\REAL-ESTATE-EDA-AND-MODEL\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Random Forest: RMSE = 716333.1612701032, R² = 0.8445726784619175
Gradient Boosting: RMSE = 637529.4597925631, R² = 0.8768887428738651


d:\dev\REAL-ESTATE-EDA-AND-MODEL\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [173]:
# Perform 5-fold cross-validation on the Random Forest model
cv_scores = cross_val_score(RandomForestRegressor(random_state=42), X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Convert the negative MSE to RMSE
cv_rmse = np.sqrt(-cv_scores)

# Print the cross-validation RMSE scores and their average
print(f"Cross-Validation RMSE scores: {cv_rmse}")
print(f"Mean CV RMSE: {np.mean(cv_rmse)}")

Cross-Validation RMSE scores: [11956275.69652412  1044499.55382542  1024126.14550588   482869.28329701
   626053.81351854]
Mean CV RMSE: 3026764.8985341927
